In [2]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


1. Загрузите данные из файла data-logistic.csv. Это двумерная выборка, целевая переменная на которой принимает значения -1 или 1.
2. Убедитесь, что выше выписаны правильные формулы для градиентного спуска. Обратите внимание, что мы используем полноценный градиентный спуск, а не его стохастический вариант!
3. Реализуйте градиентный спуск для обычной и L2-регуляризованной
(с коэффициентом регуляризации 10) логистической регрессии. Используйте длину шага k=0.1. В качестве начального приближения
используйте вектор (0, 0).
4. Запустите градиентный спуск и доведите до сходимости (евклидово
расстояние между векторами весов на соседних итерациях должно быть не больше 1e-5). Рекомендуется ограничить сверху число
итераций десятью тысячами.
5. Какое значение принимает AUC-ROC на обучении без регуляризации и при ее использовании? Эти величины будут ответом на
задание. В качестве ответа приведите два числа через пробел. Обратите внимание, что на вход функции roc_auc_score нужно подавать оценки вероятностей, подсчитанные обученным алгоритмом.
Для этого воспользуйтесь сигмоидной функцией: a(x) = 1/(1 +
exp(−w1x1 − w2x2)).

In [29]:
import pandas as pd
import numpy as np
from typing import Tuple
from sklearn.metrics import roc_auc_score

In [ ]:
!ls /content/gdrive/'MyDrive'/ML/

In [5]:
data = pd.read_csv('/content/gdrive/MyDrive/ML/data-logistic.csv', header=None)

In [6]:
data.head(5)

,0,1,2
0,-1,-0.663827,-0.138526
1,1,1.994596,2.468025
2,-1,-1.247395,0.749425
3,1,2.309374,1.899836
4,1,0.849143,2.407750


In [16]:
y = data.loc[:,0]
X = data.loc[:,1:]

In [19]:
def w1_grad(X: pd.DataFrame, y: pd.Series, w1: float, w2: float, k: float, C: float) -> float:
    sum = 0
    l = len(y)
    for i in range(l):
        sum += y[i] * X[1][i] * (1 - 1/(1 + np.exp(-y[i] * (w1 * X[1][i] + w2 * X[2][i]))))

    return w1 + (k * sum) / l - w1 * k * C


def w2_grad(X: pd.DataFrame, y: pd.Series, w1: float, w2: float, k: float, C: float) -> float:
    sum = 0
    l = len(y)
    for i in range(l):
        sum += y[i] * X[2][i] * (1 - 1 / (1 + np.exp(-y[i] * (w1 * X[1][i] + w2 * X[2][i]))))

    return w2 + (k * sum) / l - w2 * k * C

In [43]:
def gradient(X: pd.DataFrame, y: pd.Series, w1: float=0.0, w2: float=0.0,
             k: float=0.1, C: float=0.0, precisison: float=1e-5, iterations: int=10000) -> Tuple[float, float]:
    count = 0
    for i in range(iterations):
        count += 1
        w1_pred = w1
        w2_pred = w2
        w1 = w1_grad(X, y, w1, w2, k, C)
        w2 = w2_grad(X, y, w1, w2, k, C)
        if ((w1_pred - w1) ** 2 + (w2_pred - w2) ** 2) <= precisison:
            break
    
    
    print(f'итераций: {count}')
    return w1, w2

In [23]:
def a(X: pd.DataFrame, w1: float, w2: float) -> pd.Series:
    return 1.0/(1.0 + np.exp(-w1 * X[1] - w2 * X[2]))

In [44]:
w1, w2 = gradient(X, y)
w1_L2, w2_L2 = gradient(X, y, C=10.0)
print(w1, w2)
print(w1_L2, w2_L2)

итераций: 21
итераций: 3
0.21863379967073307 0.1317909454295162
0.02879214117828812 0.02481195193696825


In [38]:
predict = a(X, w1, w2)
predict_L2 = a(X, w1_L2, w2_L2)

In [40]:
print(roc_auc_score(y, predict))
print(roc_auc_score(y, predict_L2))

0.9362857142857142
0.9363809523809522


точность без/с L2 нормализацей сопоставима, число итераций: \
 без нормализации - 21 \
 с L2 нормализацией - 3

6. Попробуйте поменять длину шага. Будет ли сходиться алгоритм,
если делать более длинные шаги? Как меняется число итераций
при уменьшении длины шага?
7. Попробуйте менять начальное приближение. Влияет ли оно на чтонибудь?

In [51]:
w1, w2 = gradient(X, y, k=4) # при k > 4 алгоритм не сходится

итераций: 10000


In [52]:
for i in [1.0, 0.1, 0.01, 0.001, 0.0001]:
    w1, w2 = gradient(X, y, k=i)

итераций: 9
итераций: 21
итераций: 17
итераций: 1
итераций: 1


при уменьшении длины шага число итераций снижается

In [53]:
w1, w2 = gradient(X, y, w1=1.0, w2=2.0)

итераций: 91


In [54]:
w1, w2 = gradient(X, y, w1=10.0, w2=20.0)

итераций: 483


In [55]:
w1, w2 = gradient(X, y, w1=-10.0, w2=-20.0)

итераций: 362


при изменении начальных приближений число итераций увеличивается

In [60]:
w1, w2 = gradient(X, y, w1=-10.0, w2=20.0, k=0.0001)

итераций: 1
